# A specutils example for GBspec

This version uses the SpectrumList container. Could also consider a list of Spectrum1D or a SpectrumCollection.

This should reproduce Example 1 from the GBTIDL manual.

## SDFITS

We begin by disecting the typical SDFITS file, starting with raw plotting



In [ ]:
# %matplotlib inline
%matplotlib

from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from specutils import Spectrum1D, SpectrumList

A useful stats routine so we can regress if we see the same data

In [ ]:
def my_stats(label,data):
    """
    for regression
    """
    mean = data.mean()
    rms  = data.std()
    dmin = data.min()
    dmax = data.max()
    print("%s  %s %s %s %s" %  (label,repr(mean),repr(rms),repr(dmin),repr(dmax)))

In [ ]:
fname = 'ngc5291.fits'
row   = 0

In [ ]:
hdu = fits.open(fname)
header2 = hdu[1].header
data2   = hdu[1].data
print("%d rows" % len(data2))

In [ ]:
flux = data2[row]['DATA']  
nchan = len(flux)
chans  = np.arange(nchan)
print("%d channels" % nchan)
#
my_stats('STATS for row %d:' % row,flux)

In [ ]:
plt.plot(chans,flux);

### How about Panda DataFrame's ?


In [ ]:
from astropy.table import Table
import pandas

t= Table.read(fname,format='fits') 
if 'TDIM7' in t.colnames:
    print("Expect trouble, there is TDIM7")
#t.meta
df = t.to_pandas()
 
# problem is that TDIM7 = '(32768,1,1,1,)' so we should comment out TDIM7 -> XTDIM7 or remove column
# hmm, still doesn't work

### Adding astropy units to the X and Y axis

In [ ]:
crval1 = data2[row]['CRVAL1']
cdelt1 = data2[row]['CDELT1']
crpix1 = data2[row]['CRPIX1']
freq0 = data2[row]['RESTFREQ']
freq = crval1 + (np.arange(1,nchan+1) - crpix1) * cdelt1 / 1e9
#
flux = flux * u.Unit("K")
freq = freq * u.Unit("GHz")

In [ ]:
# even though we've attached astropy units, they still work in matplotlib plotting
plt.plot(freq,flux);

### Turning into a simple Spectrum1D

In [ ]:
# now make a Spectrum1D to work from
spec = Spectrum1D(spectral_axis=freq, flux=flux)

In [ ]:
f, ax = plt.subplots()  
ax.step(spec.spectral_axis, spec.flux);
# darn, the spectral axis still has 1e9 units

## Designing our own reader in specutils

In [ ]:
from specutils.io import get_loaders_by_extension

loaders = get_loaders_by_extension('fits')
print(loaders)

In [ ]:
from astropy.nddata import StdDevUncertainty
from astropy.table import Table
from astropy.units import Unit
from astropy.wcs import WCS

from specutils.io.registers import data_loader
from specutils import Spectrum1D


We need a list of FITS keywords and FIELD names from the BINTABLE that are going to be the meta-data
associated with each spectrum.


In [ ]:
# just a few important ones for now
sdfits_headers = ['SCAN', 'PROCSEQN', 'CAL', 'OBJECT','SAMPLER', 'TSYS']



We are registering a special fits reader for the spectra, and use a SpectrumList.


In [ ]:
def identify_sdfits(origin, *args, **kwargs):
    print("IDENTIFY_SDFITS")
    try:
        with fits.open(args[0]) as hdulist:
            extname = hdulist[1].header['EXTNAME']
            if extname == 'SINGLE DISH':
                print("Hurray, we have SDFITS")
                return True
            else:
                print("Warning, skipping extname %s" % extname)
                return False
    except Exception:
        return Falseflux + sl2[i].flux

    

In [ ]:
@data_loader("sdfits", identifier=identify_sdfits, dtype=SpectrumList, extensions=['fits'])
def sdfits_loader(file_name, spectral_axis_unit=None, **kwargs):

    spectra = []
    with fits.open(file_name, **kwargs) as hdulist:
        header1= hdulist[0].header        
        header2= hdulist[1].header
        data   = hdulist[1].data
        nrow   = len(data)
        nchan  = 0
        for i in range(nrow):
            sp = data[i]['DATA']
            if nchan==0:
                nchan = len(sp)     # every spectrum in SDFITS has the same length
            crval1  = data[i]['CRVAL1']
            cdelt1  = data[i]['CDELT1']
            crpix1  = data[i]['CRPIX1']
            ctype1  = data[i]['CTYPE1']     # 'FREQ-OBS' to 'FREQ'; assuming SPECSYS='TOPOCENT'
            restfrq = data[i]['RESTFREQ']
            cunit1  = 'Hz'
            crval2  = data[i]['CRVAL2']
            crval3  = data[i]['CRVAL3']
            ctype2  = data[i]['CTYPE2']
            ctype3  = data[i]['CTYPE3']
            if ctype1 == 'FREQ-OBS': ctype1  = 'FREQ'
            wcs = WCS(header={'CDELT1': cdelt1, 'CRVAL1': crval1, 'CUNIT1': cunit1,
                              'CTYPE1': ctype1, 'CRPIX1': crpix1, 'RESTFRQ': restfrq,
                              'CTYPE2': ctype2, 'CRVAL2': crval2,
                              'CTYPE3': ctype3, 'CRVAL3': crval3})
                              

            meta = {}
            if False:
                # adding the actual FITS headers is for debugging, but not in production mode
                meta['header1'] = header1
                meta['header2'] = header2
            if True:
                for key in sdfits_headers:
                    if key in header1:
                        meta[key] = header1[key]
                    elif key in header2:
                        meta[key] = header2[key]
                    else:
                        meta[key] = data[i][key]    # why doesn't       key in data[i]    work?
                # add our row counter
                meta['_row'] = i
                        
        
            sp = sp * Unit('K')
            spec = Spectrum1D(flux=sp, wcs=wcs, meta=meta)
            spectra.append(spec)
            
    return  SpectrumList(spectra)

Now we are ready for some action!


In [ ]:
sl1 = SpectrumList.read(fname, format="sdfits")
nsp = len(sl1)
print("Found %d spectra" % nsp)
for i in range(2):
    my_stats("test",sl1[i].flux.value)

In [ ]:
a=sl1[0]-sl1[1]
b=sl1[2]-sl1[3]
print(a)
print(b)
try:
    c = a + b
except:
    print("Cannot combine")
a1 = a / sl1[0]
print(a1)
# cannot do a+b, spectral axis is TOPO
# does this imply that if our ephemeris isn't good enough, they will not align in doppler space
# and thus refuse to be combined?

In [ ]:
# lets look at the meta data where we know there are changes
# there are 8 scans of 11 integrations each, two polarizations and the on/off cal cycle
for i in [0,1,22,44,88]:
    print(i,sl1[i].meta)


In [ ]:
spectra = []
for i in range(0,nsp,2):
    spon  = sl1[i]         
    spoff = sl1[i+1]
    tsys  = sl1[i].meta['TSYS']
    sp = (spon-spoff)/spoff.flux.value        # trick: otherwise it's loosing the units here
    sp.meta = spon.meta
    #sp.flux *= tsys           # this is silly, can't modify the flux.....
    spectra.append(sp)
sl2 =  SpectrumList(spectra)   

In [ ]:
print(sp)
a=sp.multiply(2*u.Unit('mJy'))   
print(sp)
print(a)
a = a * (2*u.Unit('mJy'))
print(a)


In [ ]:
# just for fun, plot the lst On-Off
spd = spon - spoff
plt.plot(sl2[0].spectral_axis, spd.flux);

In [ ]:
for i in [0,1,11,22]:
    print(i,sl2[i].meta)

In [ ]:
for i in range(len(sl2)):flux + sl2[i].flux
    plt.plot(sl2[i].spectral_axis, sl2[i].flux)

In [ ]:
# this is wrong, but i don't know why yet, noise seems to high too
# something more must be going on in the getps() procedure
# I'm averaging 8 scans here, but getps() example suggests only the odd ones
sl3 = []
flux = sl2[0].flux * 0.0
for i in range(len(sl2)):
    flux = flux + sl2[i].flux
flux = flux / len(sl2)   
plt.plot(sl2[0].spectral_axis,flux);
        

In [ ]:
sp = sl2[0]
sp = sp.multiply(0*u.Unit('Jy'))     # the unit doesn't appear to be import, but it needs a unit
print(sp)

In [ ]:
for i in range(len(sl2)):
    sp = sp.add(sl2[i])